In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Activation
from keras.layers import Conv2D, MaxPool2D

In [2]:
(X_train, y_train), (X_test, y_test) = mnist.load_data('/tmp/mnist.npz')
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255.0
X_test /= 255.0
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [3]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(128, activation='relu'))

model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [4]:
model.fit(X_train, y_train_cat, batch_size=128,
          epochs=2, verbose=0, validation_split=0.3)
print(model.summary())
model.evaluate(X_test, y_test_cat)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
module_wrapper (ModuleWrappe (None, 26, 26, 32)        320       
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 13, 13, 32)        0         
_________________________________________________________________
activation (Activation)      (None, 13, 13, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 5408)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               692352    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
__________________________________________________

[0.0653628259897232, 0.9794999957084656]

#### Uitleg van de code
Er wordt een sequentieel model aangemaakt. 
Hieraan worden 6 lagen toegevoegd.
- een conv2d laag. Dit is een convolutie kernel in dit geval op een 32x32 image met een grootte van 3x3.
  de input_shape bestaat uit een 28x28 image
  
  
- een Max2DPool met een pool_size van 2x2. Dit zorgt voor data reductie. De operatie pakt een stuk van 2x2 uit de image en gooit alles weg behalve de hoogste waarde die gevonden wordt. Deze datareductie zal verder weinig van invloed zijn op de uiteindelijke classificatie maar er is wel een hoop minder data te verwerken.


- een Activation functie van het type Relu. Dit is een snelle functie voor het vinden van het lokale minimum van de loss functie.


- een flatten operatie om de voorgaande lagen samen te voegen.


- een dense layer van 128 tensors met de relu functie


- een dense layer van 10 tensors met de softmax functie. Hiermee kan het resultaat in percentage tussen 0 en 1 worden uitgedrukt.


Vervolgens wordt het model gebouwd met als loss functie de categorical crossentropy en als optimizer rmsprop.
Er wordt gekeken naar de accuracy metric voor dit model.


#### Verklaar alle waardes in de summary tabel
De kolom Param # bevat het aantal parameters.

Voor dense layers geldt: `output_size * (input_size + 1) == number_parameters`\
Voor conv layers geldt: `output_channels * (input_channels * window_size + 1) == number_parameters`

- Voor rij 1: `32 * (1* (3*3)+1) = 320` 
- Voor rij 5: `128 * (5408 * 1 + 1) = 692352`
- Voor rij 6: `10 *(128 * 1 + 1) = 1290`
- Total Parameters: `320 + 692352 + 1290 = 693962`

Bron: https://stackoverflow.com/a/56534957

#### Welke conclusie trek je op basis van de output van `model.evaluate()`?
De evaluate functie print de loss en de accuracy van het model uit.
De accuracy staat op 0.986. Het model is dus erg precies.
De loss staat op 0.052. Dit zit erg dicht bij 0. Daarom is de voorspelling van het model waarschijnlijk goed.